Simulation and PINN Model Development for 3D cube 

Importing the libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import csv
from sklearn import svm
import pandas as pd
import itertools
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

Defining the physical constants

Defining the domain